# Face recognition with ORL

In [190]:
import numpy as np
from scipy import ndimage
from matplotlib import pyplot as plt
from PIL import Image
import os
import dlib
import cv2
import random

In [191]:
def randomVyb():
    testVyb = np.zeros(80)
    trainVyb = [0]*400
    trainVybNp = np.zeros(400)
    vyb = np.zeros(400)
    rand = np.zeros(2)
    v = 0
    
    for i in range (0, 400):
        vyb [i] = i
        trainVyb [i] = i
        
    for k in range (0, 399, 10):
        for i in range (0, 2,):
            rand [0] = random.randint (0, 4)
            rand [1] = random.randint (5, 9)
            testVyb[v] = k + rand [i]
            v += 1

    for g in range (399, -1, -1):
        for j in range (0, 80):
            if (vyb [g] == testVyb[j]):
                trainVyb.remove(trainVyb[g]) 
                
    trainVybNp = np.array(trainVyb)            
    
    return  trainVybNp, testVyb

In [192]:
#randomVyb()

In [193]:
dsFolderPath = "ds_faces" 
rows = 0 
numberPhotoOnePerson = 0
digitalSurname = ""
photoname = ""
mmds = ""

In [194]:
def pathesOfImagesFromDs(dsFolderPath):
    global rows
    global numberPhotoOnePerson
    listOfSubfolders = os.listdir(dsFolderPath)
    photos = list()
    for i in listOfSubfolders:
        listOfImagesInCurrentSubfolder = os.listdir(dsFolderPath + "/" + i)
        for j in listOfImagesInCurrentSubfolder:            
            photos.append(dsFolderPath + "/" + i + "/" + j)
    rows = len(photos)
    photos.sort()
    numberPhotoOnePerson = len (listOfSubfolders)
    return rows, numberPhotoOnePerson, photos

In [195]:
#pathesOfImagesFromDs(dsFolderPath)

In [196]:
def initDigitalSurname():
    global digitalSurname
    global number_ds
    number_ds = numberPhotoOnePerson
    numberDigitalSurname = rows / number_ds 
    digitalSurname = [0] * rows
    for i in range (0, number_ds, 1):
        for j in range (0, int(numberDigitalSurname), 1):
            digitalSurname [int(str(i) + str(j))] = i + 1
    return number_ds, numberDigitalSurname, len(digitalSurname), digitalSurname

In [197]:
#initDigitalSurname()

In [198]:
def getSizePhoto ():
    img = np.array(pathesOfImagesFromDs(dsFolderPath)[2])
    i = 0 
    img2 = cv2.imread(img[i]) 
    h = img2.shape[0]
    w = img2.shape[1]
    c = img2.shape[2]
    n = h * w
    m = rows    
    return m, n

In [199]:
#getSizePhoto ()

In [200]:
def create_MMDS():
    m = getSizePhoto ()[0]
    n = getSizePhoto ()[1]
    mmds = np.zeros((m,n))
    return mmds       

In [201]:
def parsePhoto (i):
    im=np.array(Image.open(pathesOfImagesFromDs(dsFolderPath)[2][i]))
    im=im.reshape(1,im.shape[0]*im.shape[1])    
    return im

In [202]:
def create_MMDS():
    m = getSizePhoto ()[0]
    n = getSizePhoto ()[1]
    global mmds
    mmds = np.zeros((m,n)) 
    for i in range (0, m):
        mmds [i] = (parsePhoto (i))/255 
    return mmds

In [203]:
#create_MMDS()

In [204]:
def numberSplitMMDS (numberTrain):
    numberTest = 10 - numberTrain
    return numberTrain, numberTest

In [205]:
def splitMMDS ():
    n = getSizePhoto ()[1] 
    m = getSizePhoto ()[0]
    i_trainMMDS = 0
    i_testMMDS = 0

    tM = 8
    mmds = create_MMDS()
    testM = int(initDigitalSurname()[1]) - tM
    trainMMDS = np.zeros(((tM * number_ds), n)) 
    trainDigitalSurname = list()
    testMMDS = np.zeros(((testM * number_ds), n)) 
    testDigitalSurname = list()
    randomV = randomVyb()
    
    for i in range (0, 400,):      
        for v in range (0, 320,):           
            if (i == randomV[0][v]):   
                trainMMDS[i_trainMMDS,:] = mmds[i,:]
                i_trainMMDS += 1       
                trainDigitalSurname.append(digitalSurname[i])
                
    for i in range (0, 400,):       
        for v in range (0, 80,):          
            if (i == randomV[1][v]):   
                testMMDS[i_testMMDS,:] = mmds[i,:]    
                i_testMMDS += 1    
                testDigitalSurname.append(digitalSurname[i])
    return trainMMDS, trainDigitalSurname, testMMDS, testDigitalSurname

In [206]:
#splitMMDS ()

In [208]:
def evklidDistanse (vector1, vector2):
    return np.linalg.norm(vector2 - vector1)

In [209]:
def minDist():
    mmds1 = splitMMDS()[0] 
    trainDS = splitMMDS()[1] 
    mmds2 = splitMMDS()[2] 
    testDS = splitMMDS()[3] 
    acc = 0 
    vector1 = np.zeros (len(mmds1)) 
    vector2 = np.zeros (len(mmds2)) 
    iTest = 0
    
    for i in range (0, len(mmds2), 1): 
        iTest += 1
        minDist = 100
        for j in range (0, len(mmds1),): х 
            vector1 = mmds1[j] 
            vector2 = mmds2[i] 
            evk = evklidDistanse (vector1, vector2)
            if (evk < minDist):
                minDist = evk
                jv = j
                evk = 0            
        if (testDS[i] == trainDS[jv]):
            acc += 1
    
    return ((acc / iTest) * 100)

In [210]:
print ("accurancy = ", minDist(), "%")

Количество эталонных фамилий -  320
Количество тестовых фамилий -  80
accurancy =  100.0 %
